In [10]:
import os

# Affiche le répertoire courant
current_directory = os.getcwd()
print("Répertoire courant :", current_directory)

Répertoire courant : /Users/NDAO028930/Documents/docs/mmi/parcours data engineer/projets/JobMarket/processing


In [8]:
import sys
sys.path.append('../')

import pandas as pd
from tqdm import tqdm
tqdm.pandas()  # Initialiser tqdm pour pandas
from common.utils import translate_french_to_english
from common.utils import clean_text
import pickle
from datetime import datetime

ModuleNotFoundError: No module named 'configs.conf'

In [ ]:
#jobs_adzuna= pd.read_pickle('../outputs/raw/jobs_adzuna_sept.pkl')
jobs_adzuna= pd.read_pickle('../outputs/raw/jobs_adzuna_us_nov.pkl')
jobs_themuse = pd.read_pickle('../outputs/raw/jobs_muse_sept.pkl')
jobs_wttj = pd.read_pickle('../outputs/raw/jobs_wttj_aout.pkl')
#jobs_linkedin = pd.read_pickle('../outputs/raw/jobs_linkedin_sept.pkl')
jobs_linkedin_1 = pd.read_pickle('../outputs/raw/jobs_linkedin_sept.pkl')
jobs_linkedin_2 = pd.read_pickle('../outputs/raw/jobs_linkedin_nov.pkl')
jobs_linkedin_3 = pd.read_pickle('../outputs/raw/jobs_linkedin_nov_5.pkl')
jobs_linkedin_4 = pd.read_pickle('../outputs/raw/jobs_linkedin_nov_10.pkl')
jobs_linkedin_5 = pd.read_pickle('../outputs/raw/jobs_linkedin_nov_15.pkl')
jobs_linkedin_dup = pd.concat([jobs_linkedin_1,jobs_linkedin_2,jobs_linkedin_3,jobs_linkedin_4,jobs_linkedin_5], ignore_index=True)
jobs_linkedin = jobs_linkedin_dup.drop_duplicates()



links_linkedin = pd.read_pickle('../outputs/raw/company_links_nov_10.pkl')


companies_linkedin_1 = pd.read_pickle('../outputs/raw/companies_linkedin_nov.pkl')
companies_linkedin_2 = pd.read_pickle('../outputs/raw/companies_linkedin_nov_5.pkl')
companies_linkedin_3 = pd.read_pickle('../outputs/raw/companies_linkedin_nov_10.pkl')
companies_linkedin_4 = pd.read_pickle('../outputs/raw/companies_linkedin_nov_15.pkl')
companies_linkedin_dup = pd.concat([companies_linkedin_1,companies_linkedin_2,companies_linkedin_3,companies_linkedin_4], ignore_index=True)
companies_linkedin = companies_linkedin_dup.drop_duplicates()


#test = jobs_linkedin['company_link'].unique
#print(test)

#jobs_linkedin.head()
companies_linkedin.head()


print(len(jobs_linkedin))
#14650 for United States jobs
#1634 in jobs_linkedin_nov_5.pkl
#2078
#7270 all

print(len(companies_linkedin))
#69 in  companies_linkedin_nov_5.pkl
#83 in companies_linkedin_nov_10.pkl
#94 in companies_linkedin_nov_15.pkl
#251 all


companies_linkedin.head()




7270
251


,company_name,company_description,company_website,company_industries,company_size,company_associated_members,company_location,company_creation_date,company_specialities,source,ingest_date
0,U.S. House of Representatives,The U.S. House of Representatives is one of tw...,http://www.house.gov/,Legislative Offices,"10,001+ employees",LinkedIn members who’ve listed U.S. House of R...,"Washington, DC",None,None,linkedin,2024-11-07 18:17:14.373196
1,Relate Search,"Welcome to Relate Search, the purpose-driven s...",https://www.relatesearch.com/,Staffing and Recruiting,11-50 employees,LinkedIn members who’ve listed Relate Search a...,"Cincinnati , OH",2023,"Recruiting, Executive Search, Human Resources,...",linkedin,2024-11-07 18:17:14.373196
2,Avispa,"At Avispa, we ensure our contractors are as sa...",http://www.avispatechnology.com,Staffing and Recruiting,51-200 employees,LinkedIn members who’ve listed Avispa as their...,"Truckee, California",2023,"Recruiting, Executive Search, Human Resources,...",linkedin,2024-11-07 18:17:14.373196
3,clearer.io,"At Clearer.io, we’re on a mission to simplify ...",https://www.clearer.io/,Software Development,201-500 employees,LinkedIn members who’ve listed clearer.io as t...,"Boston, Massachusetts",2011,"Workforce Management Services, Flexible Worker...",linkedin,2024-11-07 18:17:14.373196
4,Cigna Healthcare,We are a health benefits provider that advocat...,Visit website,Hospitals and Health Care,"10,001+ employees",LinkedIn members who’ve listed Cigna Healthcar...,"Bloomfield, CT",2011,"Workforce Management Services, Flexible Worker...",linkedin,2024-11-07 18:17:14.373196


In [12]:
jobs_adzuna_selected = jobs_adzuna[['title','description','contract_type', 'contract_time',
'company.display_name', 'created','location.display_name','category.label', 'salary_max' ,'salary_min', 'location.area',
'redirect_url','source']].rename(columns={'title': 'job_title', 'description': 'job_description',
'company.display_name': 'company_name','created': 'publication_datetime','location.display_name': 'location','category.label': 'category',
'location.area': 'locations_company','redirect_url': 'offer_link'})

jobs_adzuna_selected['publication_date'] = pd.to_datetime(jobs_adzuna_selected['publication_datetime']).dt.strftime("%Y-%m-%d")

jobs_adzuna_selected['publication_time'] = jobs_adzuna_selected['publication_datetime'].apply(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%SZ").time())
jobs_adzuna_selected['country']='US'
jobs_adzuna_selected['industry']='not available'
jobs_adzuna_selected['currency']='dollars'

jobs_adzuna_selected['locations_company'] = jobs_adzuna_selected['locations_company'].astype(str)

jobs_adzuna_selected['locations_company'] = jobs_adzuna_selected['locations_company'].str.replace(r'[\[\]]', '', regex=True)
jobs_adzuna_selected.head()

#companies['contents'] = companies['description'].apply(clean_text)



,job_title,job_description,contract_type,contract_time,company_name,publication_datetime,location,category,salary_max,salary_min,locations_company,offer_link,source,publication_date,publication_time,country,industry,currency
0,Internal Medicine Physician,Are you a dedicated Internal Medicine Physicia...,permanent,NaN,AnyPlace MD,2024-09-12T04:47:34Z,"Fort Douglas, Salt Lake County",Healthcare & Nursing Jobs,225000.0,0.0,"'US', 'Utah', 'Salt Lake County', 'Fort Douglas'",https://www.adzuna.com/details/4861152531?utm_...,adzuna,2024-09-12,04:47:34,US,not available,dollars
1,Customer Service Representative,"As a Customer Service Representative, you'll b...",permanent,full_time,Ada Business Solutions,2024-06-14T09:33:24Z,"Milwaukie, Clackamas County",Customer Services Jobs,45600.0,26400.0,"'US', 'Oregon', 'Clackamas County', 'Milwaukie'",https://www.adzuna.com/details/4736389699?utm_...,adzuna,2024-06-14,09:33:24,US,not available,dollars
2,Medical Director / Physician Opportunity on La...,Are you a Physician ready for a fresh start in...,permanent,NaN,AnyPlace MD,2024-09-04T04:44:03Z,"Racine, Racine County",Healthcare & Nursing Jobs,300000.0,225000.0,"'US', 'Wisconsin', 'Racine County', 'Racine'",https://www.adzuna.com/details/4851460203?utm_...,adzuna,2024-09-04,04:44:03,US,not available,dollars
3,Customer Service Representative,"As a Customer Service Representative, you'll b...",permanent,full_time,Ada Business Solutions,2024-09-02T12:23:30Z,"West Linn, Clackamas County",Customer Services Jobs,45600.0,26400.0,"'US', 'Oregon', 'Clackamas County', 'West Linn'",https://www.adzuna.com/details/4849468031?utm_...,adzuna,2024-09-02,12:23:30,US,not available,dollars
4,Portland Medical Director / Physician Opportunity,We are seeking a dedicated Physician to serve ...,permanent,NaN,AnyPlace MD,2024-09-04T04:44:03Z,"Clackamas, Clackamas County",Healthcare & Nursing Jobs,300000.0,225000.0,"'US', 'Oregon', 'Clackamas County', 'Clackamas'",https://www.adzuna.com/details/4851460201?utm_...,adzuna,2024-09-04,04:44:03,US,not available,dollars


In [13]:
jobs_theMuse_selected = jobs_themuse[['name','contents','type', 'company.name',
'publication_date', 'locations','categories','refs.landing_page']].rename(columns={'name': 'job_title', 'contents': 'job_description',
'type': 'contract_type','company.name': 'company_name','publication_date': 'publication_datetime','locations': 'location',
'categories': 'category','refs.landing_page': 'offer_link'})

jobs_theMuse_selected['publication_date'] = pd.to_datetime(jobs_adzuna_selected['publication_datetime']).dt.strftime("%Y-%m-%d")
jobs_theMuse_selected['publication_time'] = jobs_theMuse_selected['publication_datetime'].apply(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%SZ").time())

jobs_theMuse_selected['country']='a travailler'
jobs_theMuse_selected['industry']='not available'
jobs_theMuse_selected['currency']='a travailler'
jobs_theMuse_selected['contract_time']='not available'
jobs_theMuse_selected['locations_company']='not available'
jobs_theMuse_selected['salary_max']=0
jobs_theMuse_selected['salary_min']=0

jobs_theMuse_selected['location'] = jobs_theMuse_selected['location'].astype(str)
jobs_theMuse_selected['location'] = jobs_theMuse_selected['location'].str.replace(r'[\[\]]', '', regex=True)

jobs_theMuse_selected['job_description'] = jobs_theMuse_selected['job_description'].apply(clean_text)

jobs_theMuse_selected["source"] = "TheMuse"

jobs_theMuse_selected.head()


,job_title,job_description,contract_type,company_name,publication_datetime,location,category,offer_link,publication_date,publication_time,country,industry,currency,contract_time,locations_company,salary_max,salary_min,source
0,Freileitungsmonteur (w/m/d) Fahrleitungs- und ...,<p><strong>Gemeinsam machen wir den Unterschie...,external,Siemens,2023-01-24T00:42:10Z,"'Bad Mergentheim, Germany'",Computer and IT,https://www.themuse.com/jobs/siemens/freileitu...,2024-09-12,00:42:10,a travailler,not available,a travailler,not available,not available,0,0,TheMuse
1,Mechanical Assembler - 1st Shift,<p><strong>Mechanical Assembler</strong><br><b...,external,Siemens,2023-04-26T23:41:35Z,"'Clayton, NC'",Software Engineering,https://www.themuse.com/jobs/siemens/mechanica...,2024-06-14,23:41:35,a travailler,not available,a travailler,not available,not available,0,0,TheMuse
2,Pharmacy Technician,<p>Bring your heart to CVS Health. Every one o...,external,CVS Health,2024-06-06T12:25:46Z,"'Rockville, MD'",Healthcare,https://www.themuse.com/jobs/cvshealth/pharmac...,2024-09-04,12:25:46,a travailler,not available,a travailler,not available,not available,0,0,TheMuse
3,Sr. Systems Specialist - Smart Buildings / Aut...,<p><strong>Who designs your future? You do.</s...,external,Siemens,2023-05-03T23:45:16Z,"'Waterloo, IA'",Design and UX,https://www.themuse.com/jobs/siemens/sr-system...,2024-09-02,23:45:16,a travailler,not available,a travailler,not available,not available,0,0,TheMuse
4,Pharmacy Technician Front End - Full Time,<p>Bring your heart to CVS Health. Every one o...,external,CVS Health,2024-06-06T12:33:58Z,"'Edison, NJ'",Healthcare,https://www.themuse.com/jobs/cvshealth/pharmac...,2024-09-04,12:33:58,a travailler,not available,a travailler,not available,not available,0,0,TheMuse


In [ ]:
Muse	Linkedin	wttj	adzuna	Champ
Name 	title 	title	title 	job_title
contents 	rien	 contents	description 	job_description
Type 		 employmentType	contract_type 	contract_type
rien	Employement type 	rien	contract_time 	contract_time
company.name 	company 	 company	Company. display_name 	company_name
publication_date 	rien	 date_public	created 	publication_datetime
				publication_date
rien	Posted time 	rien	rien	publication_time
Locations	location 	 addressLocality	Location.display_name 	location
rien	rien	 addressCountry	rien	country
Categories	rien	rien	Category.label 	category
rien	Job industries 	 industry	rien	industry
rien	 salary_max	rien	salary_max 	salary_max 
rien	 salary_min	rien	salary_min 	salary_min 
				currency
rien	rien	rien	Location.are 	locations_company
Refs.landing_page 	link 	Link_job	Redirect_url 	offer_link

In [14]:
jobs_wttj_selected = jobs_wttj[['title', 'company', 'type_contract', 'contents', 'date_public',
       'employmentType','addressCountry', 'addressLocality',  'industry', 'link_job']].rename(
columns={'title': 'job_title', 'contents': 'job_description',
'employmentType': 'contract_type','company': 'company_name','date_public': 'publication_datetime','addressLocality': 'location',
' addressCountry': 'country','link_job': 'offer_link'})



jobs_wttj_selected['publication_date'] = pd.to_datetime(jobs_wttj_selected['publication_datetime']).dt.strftime("%Y-%m-%d")
jobs_wttj_selected['publication_time'] = jobs_wttj_selected['publication_datetime'].apply(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%SZ").time())
jobs_wttj_selected['industry']='not available'
jobs_wttj_selected['currency']='a travailler'
jobs_wttj_selected['contract_time']='not available'
jobs_wttj_selected['locations_company']='not available'
jobs_wttj_selected['category']='not available'

jobs_wttj_selected['salary_max']=0
jobs_wttj_selected['salary_min']=0


jobs_wttj_selected['location'] = jobs_wttj_selected['location'].astype(str)
jobs_wttj_selected['location'] = jobs_wttj_selected['location'].str.replace(r'[\[\]]', '', regex=True)

jobs_wttj_selected['job_description'] = jobs_wttj_selected['job_description'].apply(clean_text)

jobs_wttj_selected["source"] = "Wttj"

jobs_wttj_selected.head()

,job_title,company_name,type_contract,job_description,publication_datetime,contract_type,addressCountry,location,industry,offer_link,publication_date,publication_time,currency,contract_time,locations_company,category,salary_max,salary_min,source
0,Manager Conseil en transformation data (H/F),iQo,Permanent contract,<h3>🚀 <strong>iQo – le conseil accélérateur de...,2024-08-22T13:12:43Z,FULL_TIME,FR,Paris,not available,https://www.welcometothejungle.com/en/companie...,2024-08-22,13:12:43,a travailler,not available,not available,not available,0,0,Wttj
1,Consultant.e senior Stratégie & Conseil,iQo,Permanent contract,<h3><strong>🚀 iQo – le conseil accélérateur de...,2024-08-22T13:12:43Z,FULL_TIME,FR,Paris,not available,https://www.welcometothejungle.com/en/companie...,2024-08-22,13:12:43,a travailler,not available,not available,not available,0,0,Wttj
2,Consultant.e Senior Supply chain,iQo,Permanent contract,<h3><strong>🚀 iQo – le conseil accélérateur de...,2024-08-22T13:12:43Z,FULL_TIME,FR,Paris,not available,https://www.welcometothejungle.com/en/companie...,2024-08-22,13:12:43,a travailler,not available,not available,not available,0,0,Wttj
3,Consultant.e Contract Management,iQo,Permanent contract,<h3><strong>🚀 iQo – le conseil accélérateur de...,2024-08-22T13:12:43Z,FULL_TIME,FR,Paris,not available,https://www.welcometothejungle.com/en/companie...,2024-08-22,13:12:43,a travailler,not available,not available,not available,0,0,Wttj
4,Manager Conseil Services Financiers - Offre Ba...,iQo,Permanent contract,<h3><strong>🚀 iQo – le conseil accélérateur de...,2024-08-22T13:12:43Z,FULL_TIME,FR,Paris,not available,https://www.welcometothejungle.com/en/companie...,2024-08-22,13:12:43,a travailler,not available,not available,not available,0,0,Wttj


In [9]:
# Exemple de chaîne de temps
time_str = "2024-08-22T13:12:43Z"

# Conversion en objet datetime




13:12:43


In [16]:
jobs_linkedin_selected = jobs_linkedin[['title','employment_type', 'company','salary_min','salary_max',
'posted_time','location','job_industry','link']].rename(columns={'title': 'job_title', 'employment_type': 'contract_time',
'company': 'company_name','posted_time': 'publication_date','job_industry': 'industry','link': 'offer_link'})

#jobs_linkedin_selected['publication_date'] = pd.to_datetime(jobs_adzuna_selected['publication_datetime']).dt.strftime("%Y-%m-%d")
jobs_linkedin_selected['contents']='not available'
jobs_linkedin_selected['currency']='a travailler'
jobs_linkedin_selected['publication_time'] =None

jobs_linkedin_selected['contract_type']='not available'
jobs_linkedin_selected['category']='not available'
jobs_linkedin_selected['locations_company']='not available'
#jobs_linkedin_selected['location'] = jobs_linkedin_selected['location'].astype(str)
#jobs_linkedin_selected['location'] = jobs_linkedin_selected['location'].str.replace(r'[\[\]]', '', regex=True)
#jobs_linkedin_selected['job_description'] = jobs_linkedin_selected['job_description'].apply(clean_text)
jobs_linkedin_selected.head()

,job_title,contract_time,company_name,salary_min,salary_max,publication_date,location,industry,offer_link,contents,currency,publication_time,contract_type,category,locations_company
0,SVP & Chief Marketing Officer,Full-time,Opry Entertainment Group,,,2024-09-22,"Nashville, TN","Technology, Information and Internet",https://www.linkedin.com/jobs/view/svp-chief-m...,not available,a travailler,None,not available,not available,not available
1,Work From Home - Remote Customer Service Rep -...,Full-time,NoGigiddy,,,2024-09-22,"Portland, OR",Media Production,https://www.linkedin.com/jobs/view/work-from-h...,not available,a travailler,None,not available,not available,not available
2,Remote Customer Service Rep Up to 19/hour - No...,Full-time,NoGigiddy,"$39,520.00/yr","$43,129.00/yr",2024-09-22,"Atlanta, GA",Business Consulting and Services,https://www.linkedin.com/jobs/view/remote-cust...,not available,a travailler,None,not available,not available,not available
3,Graphic Designer,Full-time,Unanimous Media,"$65,000.00","$80,000.00",2024-09-22,"Los Angeles, CA",Business Consulting and Services,https://www.linkedin.com/jobs/view/graphic-des...,not available,a travailler,None,not available,not available,not available
4,Data Entry Clerk - Hybrid,Full-time,Baca Consulting Group,$13.20,$17.20,2024-09-22,"Wilbraham, MA",Architecture and Planning,https://www.linkedin.com/jobs/view/data-entry-...,not available,a travailler,None,not available,not available,not available


In [17]:
jobs1 = pd.concat([jobs_linkedin_selected, jobs_theMuse_selected], axis=0)
jobs2 = pd.concat([jobs1, jobs_adzuna_selected], axis=0)
jobs_final = pd.concat([jobs2, jobs_wttj_selected], axis=0)

#jobs = pd.concat([jobs2, jobs_wttj_selected], axis=0)

shape=jobs_final.shape
print(f"Taille du dataframe fusionné : {shape}")


Taille du dataframe fusionné : (16269, 21)


In [57]:
jobs2.to_pickle('../data/final/jobs.pkl')
